## Red Rock Canyon Data Project

Las Vegas is known for many things. The night life, the casinos, the fact that it's in a desert. I've never been the type of person that was interested in clubs or partying, and although I can understand the appeal, I'm personally interested in playing games, statistically speaking, are designed for me to lose money. However there's one thing that's underated about Las Vegas is the nature. The Las Vegas Valley is surrounded by mountains. There's hiking to do on the East, North, West, and South. Despite all of it's other more well known features, hiking is my favorite part about my home city. 

Recently I looked up a few hiking trails at Red Rock Canyon, which is the the west of Las Vegas. When looking at the trails I noticed that they had some information on the difficulty of the hikes, as well as a few other things. Me being the type of person I am (*read: Nerd*), the first thing that came to my mind was **DATA**. This blog post is going to detail the steps I went through in making my own personal data project 

First things first, as with any project, one must lay out their plan and decide what tools they're going to use. This project was pretty small so I only needed to import a handfull of librarys. The [requests](https://docs.python-requests.org/en/latest/) library.This allows users to retrieve the web pages that contain our data via http. [bs4/BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) is used to parse through these web pages and helps users find exactly what they want on a given webpage. [Pandas](https://pandas.pydata.org/docs/) is used to create and manipulate data frames. Finally, [Plotly](https://plotly.com/python/) was used for as my data visualization library of choice. 

In [55]:
from bs4 import BeautifulSoup
import pandas as pd
import plotly.express as px
import requests
import numpy as np


After importing the above libraries, my next goal was to scrap Red Rock Canyon's web pages for the information needed. As stated previously, Requests and BeautifulSoup made this fairly simple to do. 

In [56]:

#Setting user agent to prevent errors associated with "empty" user agents when doing an http request.
user_agent = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'}

#Using requests to retrieve both html pages containing the target data. 
trails_page = requests.get(url = 'https://www.redrockcanyonlv.org/lasvegas/hikes-trails/', headers = user_agent).text
trails_page2 = requests.get(url = 'https://www.redrockcanyonlv.org/lasvegas/hikes-trails/page/2/',headers = user_agent).text

#Creating BeautifulSoup objects using the html pages we requested.
trails_soup = BeautifulSoup(trails_page, 'html.parser')
trails_soup2 = BeautifulSoup(trails_page2, 'html.parser')


#Two List comprehension to create a list of "Trail Urls" from our trails_soup objects. Finds all h2 tags with the class 'the-title.
#Then uses the object created from find_all() to find all get 'href' from 'a' tags. 
page1_list = [x.find("a").get('href') for x in trails_soup.find_all('h2',{'class':'the-title'})]
page2_list = [x.find("a").get('href') for x in trails_soup2.find_all('h2',{'class':'the-title'})]


#Concatenate the two lists together to make one list of Urls.
trail_urls = page1_list + page2_list


Checking the results.

In [57]:
trail_urls

['https://www.redrockcanyonlv.org/moenkopi-loop/',
 'https://www.redrockcanyonlv.org/calico-hills/',
 'https://www.redrockcanyonlv.org/calico-tanks/',
 'https://www.redrockcanyonlv.org/turtlehead-peak/',
 'https://www.redrockcanyonlv.org/keystone-thrust/',
 'https://www.redrockcanyonlv.org/white-rock-willow-springs/',
 'https://www.redrockcanyonlv.org/grand-circle-loop/',
 'https://www.redrockcanyonlv.org/white-rock-la-madre-spring-loop/',
 'https://www.redrockcanyonlv.org/willow-springs-loop/',
 'https://www.redrockcanyonlv.org/la-madre-spring/',
 'https://www.redrockcanyonlv.org/petroglyph-wall-trail/',
 'https://www.redrockcanyonlv.org/north-peak-trail/',
 'https://www.redrockcanyonlv.org/bridge-mountain-trail/',
 'https://www.redrockcanyonlv.org/lost-creek-childrens-discovery/',
 'https://www.redrockcanyonlv.org/smyc/',
 'https://www.redrockcanyonlv.org/ice-box-canyon/',
 'https://www.redrockcanyonlv.org/dales/',
 'https://www.redrockcanyonlv.org/pine-creek-canyon/',
 'https://www.

With my list of URLs the next task was to go through each each trail url, and to scrape the neccesary information from them. Using list comprehension I created a list of BeautifulSoup objects for each url/webpage that I gathered in the trail_urls list.  

In [58]:
trail_htmls = [BeautifulSoup(requests.get(url = link ,headers = user_agent).text, 'html.parser') for link in trail_urls]

The BeautifulSoup objects that were created above contained the information I needed. The next step was to parse through each object and compile all of the data into one dataframe.

In [59]:
#Creating DataFrame

df = pd.DataFrame({'Name': [],
                   'Distance (miles)': [],
                   'Average Time (hours)':[],
                   'Difficulty':[],
                  })
#For loop to grab needed data from htmls created above.

for x in trail_htmls:
    #This grabs some metrics from the web pages and then turns those metrics into a list.
    scraped_metrics = x.find_all('div',{'class':'mk-single-content clearfix'})[0].find('p').text.split(';')

    
    #This grabs <title> of the webpage in order to get the name of the trail. 
    scraped_name = x.find_all('title')[0].get_text().split('|')[0].strip()

    #This adds the trail name to the beggining of the metrics list.
    scraped_metrics.insert(0, scraped_name) 
    
    
    
    #This adds the list of information to the dataframe as row. 
    df.loc[len(df.index)] = scraped_metrics
     

Taking a look at this data, it quickly becomes obvious that there's some inconsistent formatting going on. My next task was to clean the data in so that it can be used for analysis. 

In [60]:
df

,Name,Distance (miles),Average Time (hours),Difficulty
0,Moenkopi Loop,Distance: 2 miles,Average time: 1.5hrs,Difficulty: EASY
1,Calico Hills,Distance 2-6 miles,Average time: 1.5 – 3.5 hrs,Difficulty: MODERATE
2,Calico Tanks,Distance 2.2 miles,Average time: 2hrs,Difficulty: MODERATE – STRENUOUS
3,Turtlehead Peak,Distance 4.6 miles,Average time: 3.5-4.5hrs,Difficulty: STRENUOUS
4,Keystone Thrust,Distance 2.4 miles,Average time: 1.5hrs,Difficulty: MODERATE
5,White Rock – Willow Springs,Distance: 4 miles,Average time: 2.5hrs,Difficulty: MODERATE
6,Grand Circle Loop,Distance: 11.4 miles,Average Time: 6 hrs,Difficulty: Strenuous
7,White Rock Mountain Loop,Distance 6.2 miles,Average time: 3.5hrs,Difficulty: STRENUOUS
8,Willow Spring Loop,Distance 1.1 miles,Average time: 1.25hrs,Difficulty: EASY
9,La Madre Spring,Distance 3.6 miles,Average time: 2hrs,Difficulty: MODERATE


In [61]:
# Cleaning data.

# Grabbing only the numeric value from distance column. 
for x,y in df.iterrows():
    y[1] = y[1].split(' ')[1]
    y[1]

#Cleaning Average Time data. 

#Stripping excess information from strings and turning values into numerics. 

for x,y in df.iterrows():
    y[2] = y[2].strip().split(':')[1].split('h')[0].strip()
    y[2]=y[2].split('-')[0]


#Petroglyph Wall had it's value as '30 minutes'.

df.loc[10,'Average Time (hours)'] = .5

#Ash Spring trail had it's value as '1/2'
df.loc[30,'Average Time (hours)'] = .5


# Cleaning to difficulty row and transforming our ordinal variables into numerical data. 

for x,y in df.iterrows():
    y[3]=y[3].split(':')[1].strip().lower()
    
# For the sake of making this analysis easier, I am turning these variables into numbered strings. 
for x,y in df.iterrows():   

    if df.loc[x,'Difficulty'] == 'easy':
        df.loc[x,'Difficulty'] = '0'
    elif df.loc[x,'Difficulty'] == 'easy-moderate':
        df.loc[x,'Difficulty'] = '1'
    elif df.loc[x,'Difficulty'] == 'easy to moderate':
        df.loc[x,'Difficulty'] = '1'
    elif df.loc[x,'Difficulty'] == 'moderate':
        df.loc[x,'Difficulty'] = '2'
    elif df.loc[x,'Difficulty'] == 'moderate - strenuous':
        df.loc[x,'Difficulty'] = '3'
    elif df.loc[x,'Difficulty'] == 'strenuous':
        df.loc[x,'Difficulty'] = '4'
        
df.loc[2,'Difficulty'] = '3'

In [62]:
df['Difficulty'].unique()

array(['0', '2', '3', '4', '1'], dtype=object)

In [63]:
df.head(5)

,Name,Distance (miles),Average Time (hours),Difficulty
0,Moenkopi Loop,2,1.5,0
1,Calico Hills,2-6,1.5 – 3.5,2
2,Calico Tanks,2.2,2,3
3,Turtlehead Peak,4.6,3.5,4
4,Keystone Thrust,2.4,1.5,2


Calico hills had it's value as '1.5 – 3.5'. According to it's description on the website, this is because there are multiple different "versions" of the trial. This is the description given directly from the webpage.

> This trail runs along the base of the Calico Rocks from Calico Basin to Sandstone Quarry. Distance is variable since the trail can be accessed at either end or from either of the two Calico parking areas. A side trail runs from the fee booth parking lot and connects with this trail (2 – 6 miles, easy to moderate).

Now this issue, took much more time than I expected to, mostly due to my indecision. My original plan was to just remove the row. But then I thought that maybe I should include each different "version" of the trail while using imputation to determine each version's Difficulty rating. I initially thought this would be a great idea, but then comes the question of how we determine the Average Time for each trail, am I going to impute that too? Even more, just looking at the [map](https://www.redrockcanyonlv.org/wp-content/uploads/calico-hills.pdf), how did they manage to get a trail length of 2 miles out of any of these sections? At the end the day I decided to just remove the row.

In regards to other trails that had non-concrete Average Times (for example Bridge Mountain), I simply chose to keep the first number listed. Unlike Calico Hills, most of these other trails didn't state the reason why they would give average time data in such a way. 




In [64]:
df.drop(index=1,inplace = True)

Lastly we make sure all our continious  are turned to numerics. 

In [65]:
df['Average Time (hours)'] = pd.to_numeric(df['Average Time (hours)'])
df['Distance (miles)'] = pd.to_numeric(df['Distance (miles)'])

Now that our data is all cleaned up, lets take a look at some descriptive statistics. 

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 30
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name                  30 non-null     object 
 1   Distance (miles)      30 non-null     float64
 2   Average Time (hours)  30 non-null     float64
 3   Difficulty            30 non-null     object 
dtypes: float64(2), object(2)
memory usage: 1.2+ KB


In [67]:
df.describe()

,Distance (miles),Average Time (hours)
count,30.000000,30.000000
mean,3.603333,2.241667
std,3.529040,1.430100
min,0.200000,0.500000
25%,1.625000,1.312500
50%,2.400000,2.000000
75%,4.000000,2.500000
max,15.800000,6.000000


Looking at the above table, the first thing that jumped out of me was the large gap between our 75% percintle in Distance (miles) and our max in Distance (miles). Without even doing any additional math, it's very obvious that we have a few outliers when it comes to our Distance. Looking at a box plot, this becomes even more clear.

In [68]:
px.box(df, y="Distance (miles)")

In [69]:
df.corr()

,Distance (miles),Average Time (hours)
Distance (miles),1.000000,0.944939
Average Time (hours),0.944939,1.000000


In [70]:
px.histogram(df, x = "Distance (miles)",color_discrete_sequence=['indianred'], template = "plotly_dark" )

In [71]:
px.histogram(df, x = "Average Time (hours)", color_discrete_sequence=['indianred'], template = "plotly_dark")

In [72]:
px.histogram(df , x = "Difficulty", color_discrete_sequence=['indianred'], template = "plotly_dark")

In [84]:
px.scatter(df, x = "Distance (miles)", y = "Average Time (hours)", 
           color = "Difficulty", color_discrete_sequence = px.colors.qualitative.Set1,template = "plotly_dark")

Looking at this chart it immediatly became clear to me that the samples with a Diffuculty rating of 3 (Strenious), are much different than the trails with lower difficulty ratings. Most ofther ratings have all of their samples seemingly grouped together much more tightly. To further invertigate this, I decided to take a look at the standard deviations when looking at the samples for each individual difficulty rating.  

In [74]:
df['Difficulty'].unique()

array(['0', '3', '4', '2', '1'], dtype=object)

In [75]:
for x in sorted(df['Difficulty'].unique()):
    print('Difficulty: ' + str(x))
    print(df[df['Difficulty'] == x][['Distance (miles)','Average Time (hours)']].describe().std())

Difficulty: 0
Distance (miles)        2.184651
Average Time (hours)    2.190713
dtype: float64
Difficulty: 1
Distance (miles)        1.432100
Average Time (hours)    1.387122
dtype: float64
Difficulty: 2
Distance (miles)        2.761530
Average Time (hours)    2.931371
dtype: float64
Difficulty: 3
Distance (miles)        0.453557
Average Time (hours)    0.377964
dtype: float64
Difficulty: 4
Distance (miles)        4.369023
Average Time (hours)    1.931613
dtype: float64


In [76]:
#This is the part where I should visualize the STD's of each thingy...

In [77]:
from sklearn.cluster import KMeans

In [78]:
k_means_model = KMeans(n_clusters=5)

In [79]:
k_means_model.fit(df[['Distance (miles)','Average Time (hours)']])

KMeans(n_clusters=5)

In [91]:
k_means_model.labels_

array([0, 0, 2, 0, 2, 1, 2, 4, 2, 4, 1, 3, 4, 0, 0, 2, 0, 4, 0, 2, 0, 2,
       2, 2, 2, 0, 4, 4, 4, 4], dtype=int32)

In [90]:
px.scatter(df, x = "Distance (miles)", y = "Average Time (hours)", 
           color = pd.Series(k_means_model.labels_).astype(str), color_discrete_sequence = px.colors.qualitative.Set1,template = "plotly_dark")

In [82]:
scaler.fit()

TypeError: fit() missing 1 required positional argument: 'X'

In [ ]:
px.scatter(df, x = "Difficulty", y = "Average Time (hours)")

In [ ]:
px.scatter(df, x = "Difficulty", y = "Distance (miles)")

In [ ]:
px.box(df, y="Difficulty")

In [ ]:
px.box(df, y="Distance (miles)")

In [ ]:
px.box(df, y="Average Time (hours)")